In [162]:
import numpy as np
from PIL import Image, ImageDraw
import math

In [163]:
def conv(image, filter):
    h, w = image.shape
    k_h = int(len(filter)/2)
    k_w = int(len(filter[0])/2)
    
    # initialize image after conv
    image_conv = np.zeros((h, w))

    # padding
    padded_image = np.pad(image, ((k_h, k_h), (k_w, k_w)), mode='edge')
    
    # convolving
    for i in range(h):
        for j in range(w):
            image_conv[i][j] = np.einsum("ij, ij -> ", padded_image[i:i+(2*k_w+1), j:j+(2*k_h+1)], filter[::-1,::-1])
    
    return image_conv

def edge_detection(image, sigma):
    # gaussian filter
    def gaussian_filter(k_h, k_w, sigma):
        gaussian = np.zeros((2*k_h+1, 2*k_w+1))
        for i in range(0, 2*k_h+1):
            for j in range(0, 2*k_w+1):
                gaussian[i][j] = math.exp(-((k_h-i)**2 + (k_w-j)**2)/(2 * sigma**2))/(2 * math.pi * sigma**2)

        return gaussian

    # sobel filter
    def sobel_filter(mode):
        if mode=='X':
            return np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        if mode=='Y':
            return np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    # Non Maximal Suppresion
    def NMS(I_m, I_o):
        h, w = I_m.shape

        I_suppresed = np.zeros((h, w))
        for i in range(h):
            for j in range(w):
                magnitude = I_m[i][j]
                orientation = I_o[i][j]

                is_edge = True
                if orientation >= - 1/8 * np.pi and orientation < 1/8 * np.pi:
                    if (i-1 >=0 and I_m[i-1][j] > magnitude) or (i+1 <= h-1 and I_m[i+1][j] > magnitude):
                        is_edge = False
                elif orientation >= 1/8 * np.pi and orientation < 3/8 * np.pi:
                    if (i-1 >=0 and j+1 <= w-1 and I_m[i-1][j+1] > magnitude) or (i+1 <= h-1 and j-1 >=0 and I_m[i+1][j-1] > magnitude):
                        is_edge = False
                elif orientation >= -3/8 * np.pi and orientation < -1/8 * np.pi:
                    if (i-1 >=0 and j-1 >= 0 and I_m[i-1][j-1] > magnitude) or (i+1 <= h-1 and j+1 <= w-1 and I_m[i+1][j+1] > magnitude):
                        is_edge = False
                else:
                    if (j-1 >= 0 and I_m[i][j-1] > magnitude) or (j+1 <= w-1 and I_m[i][j+1] > magnitude):
                        is_edge = False
                
                if is_edge:
                    I_suppresed[i][j] = I_m[i][j]
                else:
                    I_suppresed[i][j] = 0
        
        return I_suppresed
        
    # gaussian blurring
    gaussian = gaussian_filter(1, 1, sigma)
    image_blurred = conv(image, filter=gaussian)    

    # gradient by X, Y
    I_x = conv(image_blurred, filter=sobel_filter(mode='X'))
    I_y = conv(image_blurred, filter=sobel_filter(mode='Y'))

    # avoid zero division
    I_y[I_y == 0] += 1e-6

    # Edge Magnitude, Orientation
    I_m = np.sqrt(I_x ** 2 + I_y ** 2)
    I_o = np.arctan(I_x / I_y)

    # Non Maximal Suppression
    I_m = NMS(I_m, I_o)

    return I_m, I_o, I_x, I_y

In [164]:
def hough_lines(image, threshold, rho_res, theta_res):
    # TODO
    # Convert PIL image to NumPy array for numeric processing
    img = np.array(image)
    height, width = img.shape
    
    # Step 1: compute the values for next steps
    # Generate a list of theta values from 0 to pi (inclusive) at the given resolution
    thetas = np.arange(0, math.pi, theta_res)
    # Compute the maximum possible rho: the image diagonal length
    diag_len = int(np.ceil(np.sqrt(height**2 + width**2)))
    # Create an array of rho values from -diag_len to +diag_len
    rhos = np.arange(-diag_len, diag_len + rho_res, rho_res)
    
    # Step 2: Find the (y, x) coordinates of all pixels where intensity >= threshold
    ys, xs = np.where(img >= threshold)
    
    # Step 3: Compute contribution using Polar form parameterization and incremented the corresponding value in the accumulator array H
    # Initialize the accumulator array H to zero
    # Rows correspond to rho indices, columns to theta indices
    H = np.zeros((len(rhos), len(thetas)), dtype=np.int32)
    # For each edge pixel, compute its contribution to all (rho, theta) pairs
    for i in range(len(xs)):
        x = xs[i]
        y = ys[i]
        for t_idx in range(len(thetas)):
            theta = thetas[t_idx]
            rho = x * math.cos(theta) + y * math.sin(theta)
            rho_idx = int(round((rho + diag_len) / rho_res))
            H[rho_idx, t_idx] += 1
    return H

In [165]:
def hough_lines_peak(H, rho_res, theta_res, num_lines):
    # TODO
    # Step 1: compute suppression windows
    # Get the size of Hough accumulator
    rows, cols = H.shape
    H_copy = H.copy()
    selected = []
    # suppression windows (~10 pixels in rho, ~10° in theta)
    # r_win: rho window size, t_win: theta window size
    r_win = max(int(10.0 / rho_res), 1)
    t_win = max(int((math.pi/18) / theta_res), 1)

    # Step 2: find the peaks in the Hough space
    # Use NMS to find the peaks in the Hough space
    # Loop to find the top num_lines peaks
    for _ in range(num_lines):
        idx = np.unravel_index(np.argmax(H_copy), H_copy.shape)
        # If the H_copy value is 0, break the loop
        # This means no more peaks are found
        if H_copy[idx] == 0:
            break
        # Get index of the peak
        r_idx, t_idx = idx
        # Append the peak to the selected list
        selected.append((r_idx, t_idx))
        # Set the peak value to 0 in H_copy
        r_min = max(r_idx - r_win, 0)
        r_max = min(r_idx + r_win, rows - 1)
        t_min = max(t_idx - t_win, 0)
        t_max = min(t_idx + t_win, cols - 1)
        # Suppress the neighborhood of the peak
        H_copy[r_min:r_max+1, t_min:t_max+1] = 0

    # Step 3: Convert the selected rho and theta indices to actual values
    # rho = rho_idx * rho_res - diag_len
    # theta = t_idx * theta_res
    line_rhos = [(r - rows//2) * rho_res for r, _ in selected]
    line_thetas = [t * theta_res for _, t in selected]
    
    # return the rho and theta values
    return line_rhos, line_thetas 

'''
def hough_lines_peak(H, rho_res, theta_res, num_lines):
    # Step 1: Find local maxima in the Hough accumulator
    # Get the size of the Hough accumulator
    rows, cols = H.shape
    # Initialize a list to store the peaks
    peaks = []
    # Iterate through every cell and test if it's a local maximum
    for r in range(rows):
        for t in range(cols):
            # Get the value of the current cell
            value = H[r, t]
            # Skip if the value is less than or equal to zero
            if value <= 0:
                continue
            is_max = True
            # Check 3x3 neighborhood
            for dr in (-1, 0, 1):
                for dt in (-1, 0, 1):
                    nr, nt = r + dr, t + dt
                    if 0 <= nr < rows and 0 <= nt < cols:
                        # If the neighbor is bigger than the current cell, it's not a peak
                        if H[nr, nt] > value:
                            is_max = False
                            break
                # If we found a larger neighbor, break out of the loop
                if not is_max:
                    break
            # If it's a peak, add it to the list
            if is_max:
                peaks.append((r, t, value))

    # Step2: Sort peaks by vote count descending order
    peaks.sort(key=lambda x: x[2], reverse=True)

    # Step 3: Select the top num_lines peaks
    line_rhos = []
    line_thetas = []
    # Select top num_lines
    # Iterate through the sorted peaks and extract rho and theta values
    for r, t, _ in peaks[:num_lines]:
        rho = (r - rows // 2) * rho_res
        theta = t * theta_res
        line_rhos.append(rho)
        line_thetas.append(theta)

    return line_rhos, line_thetas 
'''

"\ndef hough_lines_peak(H, rho_res, theta_res, num_lines):\n    # Step 1: Find local maxima in the Hough accumulator\n    # Get the size of the Hough accumulator\n    rows, cols = H.shape\n    # Initialize a list to store the peaks\n    peaks = []\n    # Iterate through every cell and test if it's a local maximum\n    for r in range(rows):\n        for t in range(cols):\n            # Get the value of the current cell\n            value = H[r, t]\n            # Skip if the value is less than or equal to zero\n            if value <= 0:\n                continue\n            is_max = True\n            # Check 3x3 neighborhood\n            for dr in (-1, 0, 1):\n                for dt in (-1, 0, 1):\n                    nr, nt = r + dr, t + dt\n                    if 0 <= nr < rows and 0 <= nt < cols:\n                        # If the neighbor is bigger than the current cell, it's not a peak\n                        if H[nr, nt] > value:\n                            is_max = False\n     

In [166]:
def hough_circles(image, threshold, radius_range, a_res, b_res, r_res):
    # TODO
    
    # Convert PIL image to NumPy array for numeric processing
    img = np.array(image)
    height, width = img.shape
    
    # Step 1: compute the values for next steps
    # Unpack the minimum and maximum radius values
    r_min, r_max = radius_range
    # Generate the discrete candidate values for circle center x (a), center y (b), and radius (r)
    a_vals = np.arange(0, width, a_res)
    b_vals = np.arange(0, height, b_res)
    r_vals = np.arange(r_min, r_max + r_res, r_res)
    
    # Step 2: Find the (y, x) coordinates of all pixels where intensity >= threshold
    # Extract the coordinates of all edge pixels (intensity >= threshold)
    ys, xs = np.where(img >= threshold)
    
    # Step 3: Compute contribution using Circle parameterization and incremented the corresponding value in the accumulator array H
    # Initialize a 3D accumulator: dimensions = (len(b_vals), len(a_vals), len(r_vals))
    H = np.zeros((len(b_vals), len(a_vals), len(r_vals)), dtype=np.int32)
    # For each edge pixel, vote for all possible circles that could pass through it
    for i in range(len(xs)):
        x = xs[i]
        y = ys[i]
        # For each radius, compute the potential center coordinates (a, b)
        for r_idx, r in enumerate(r_vals):
            # Sample angles around the circle (0 to 2π) at fixed intervals
            for theta in np.arange(0, 2 * math.pi, math.pi / 18):  # 10 degrees intervals
                a = x - r * math.cos(theta)
                b = y - r * math.sin(theta)
                
                a_idx = int(round(a / a_res))
                b_idx = int(round(b / b_res))
                
                if 0 <= a_idx < len(a_vals) and 0 <= b_idx < len(b_vals):
                    H[b_idx, a_idx, r_idx] += 1
    return H

In [167]:
def hough_circles_peak(H, radius_range, a_res, b_res, r_res, num_circles):
    # TODO
    # Step 1: Find local maxima in the Hough accumulator
    H_copy = H.copy()
    selected = []
    rows, cols, depths = H_copy.shape
    # dynamic suppression windows (~10px in a and b, ~5px in radius)
    a_win = max(int(10.0 / a_res), 1)
    b_win = max(int(10.0 / b_res), 1)
    r_win = max(int(5.0 / r_res), 1)

    # Step 2: Find the peaks in the Hough space
    for _ in range(num_circles):
        idx = np.unravel_index(np.argmax(H_copy), H_copy.shape)
        if H_copy[idx] == 0:
            break
        b_idx, a_idx, r_idx = idx
        selected.append((b_idx, a_idx, r_idx))
        # suppress neighborhood
        b0 = max(b_idx - b_win, 0)
        b1 = min(b_idx + b_win, rows - 1)
        a0 = max(a_idx - a_win, 0)
        a1 = min(a_idx + a_win, cols - 1)
        r0 = max(r_idx - r_win, 0)
        r1 = min(r_idx + r_win, depths - 1)
        H_copy[b0:b1+1, a0:a1+1, r0:r1+1] = 0

    # Step 3: Convert the selected a, b, r indices to actual values
    # convert indices back to circle parameters
    r_min, _ = radius_range
    circle_as = [a_idx * a_res for _, a_idx, _ in selected]
    circle_bs = [b_idx * b_res for b_idx, _, _ in selected]
    circle_rs = [r_min + r_idx * r_res for _, _, r_idx in selected]
    
    return circle_as, circle_bs, circle_rs

In [168]:
def draw_lines(image, line_rhos, line_thetas):
    w, h = image.size
    image_with_lines = image.copy()
    image_draw = ImageDraw.Draw(image_with_lines)  

    for i in range(len(line_rhos)):
        rho = line_rhos[i]
        theta = line_thetas[i]
    
        a = np.cos(theta)
        b = np.sin(theta)

        # Avoid zero division
        if b == 0: b += 1e-6

        m = - a / b
        c = rho / b

        if abs(m) < 1:
            x1 = 0 
            y1 = m * x1 + c
            x2 = w - 1
            y2 = m * x2 + c

            image_draw.line([(x1, y1), (x2, y2)], fill='orange', width=3)
        else:
            y1 = 0
            x1 = (y1 - c) / m
            y2 = h - 1
            x2 = (y2 - c) / m

            image_draw.line([(x1, y1), (x2, y2)], fill='orange', width=3)
    
    return image_with_lines


def draw_circles(image, circle_as, circle_bs, circle_rs):
    image_with_circles = image.copy()
    image_draw = ImageDraw.Draw(image_with_circles)  

    for a, b, r in zip(circle_as, circle_bs, circle_rs):
        image_draw.ellipse((a-r, b-r, a+r, b+r), outline='orange', fill=None, width=3)
    
    return image_with_circles

In [ ]:
######## Parameters ########
# feel free to change this
sigma=2
threshold=0.08
rho_res=2.3
theta_res=math.pi/180
num_lines=20
# new parameters
# sigma=2
# threshold=0.2
# rho_res=4
# theta_res=math.pi/180
# num_lines=20
######## Parameters ########

# image = Image.open("./example1.jpg")

image = Image.open("./00000.png")
image_grey = np.array(image.convert("L"))
image_grey = image_grey / 255.

edge_image, _, __, ___ = edge_detection(image_grey, sigma)
H = hough_lines(edge_image, threshold, rho_res, theta_res)
line_rhos, line_thetas = hough_lines_peak(H, rho_res, theta_res, num_lines)
image_with_lines = draw_lines(image, line_rhos, line_thetas)

image_with_lines.save("./00000_output.png")

In [ ]:
######## Parameters ########
# feel free to change this
sigma=2
threshold=0.1
a_res=2.0
b_res=2.0
r_res=4.0
radius_range=(120, 180)
num_circles=3
# new parameters
# sigma=2
# threshold=0.3
# a_res=2.0
# b_res=2.0
# r_res=5.0
# radius_range=(30, 180)
# num_circles=4
######## Parameters ########

# image = Image.open("./example2.jpg")

image = Image.open("./00001.png")
image_grey = np.array(image.convert("L"))
image_grey = image_grey / 255.

edge_image, _, __, ___ = edge_detection(image_grey, sigma)
H = hough_circles(edge_image, threshold, radius_range, a_res, b_res, r_res)
circle_as, circle_bs, circle_rs = hough_circles_peak(H, radius_range, a_res, b_res, r_res, num_circles)
image_with_circles = draw_circles(image, circle_as, circle_bs, circle_rs)

image_with_circles.save("./00001_output.png")